In [32]:
from dotenv import load_dotenv
load_dotenv()
import os
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec

ModuleNotFoundError: No module named 'gemini'

In [43]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=768, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [19]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Wilson',
  'subject': 'Physics',
  'stars': 5,
  'review': 'Excellent professor! She explains complex topics clearly and is always willing to help.'},
 {'professor': 'Dr. John Carter',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Very knowledgeable but sometimes goes too fast. Great lectures overall.'},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'Chemistry',
  'stars': 3,
  'review': 'The course material is challenging, but Dr. Thompson is approachable and helpful during office hours.'},
 {'professor': 'Dr. Robert James',
  'subject': 'History',
  'stars': 5,
  'review': 'Passionate about history and makes the class interesting. Highly recommended!'},
 {'professor': 'Dr. Lisa Patel',
  'subject': 'Biology',
  'stars': 2,
  'review': 'Difficult to follow lectures, but the labs are informative. Needs to improve engagement.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Economics',
  'stars': 4,
  'review': 'Great professor with practical i

In [40]:
processed_data = []
client = genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

for review in data['reviews']:
    response = genai.embed_content(model="models/text-embedding-004", content=[review['review']])
    embedding = response['embedding'][0]
    
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review['review'],
            "subject": review["subject"],
            "stars": review["stars"],
        }
    })


In [41]:
processed_data[0]

{'values': [0.04752594,
  -0.007159305,
  -0.016979557,
  0.032415815,
  0.027066039,
  -0.01550794,
  -0.019437784,
  0.039220847,
  0.02012456,
  0.020178765,
  0.081212334,
  0.05906917,
  0.0034143638,
  -0.021883648,
  -0.00022805823,
  -0.05320884,
  -0.001936145,
  0.019199232,
  -0.11336531,
  0.03750776,
  0.025799338,
  -0.05250886,
  -0.009913312,
  -0.05649952,
  0.012965148,
  -0.022399949,
  0.011219987,
  -0.04401094,
  0.058737874,
  -0.062365033,
  0.054624014,
  0.013524723,
  -0.044209573,
  -0.08900618,
  -0.03816489,
  0.043932006,
  0.024307452,
  -0.026343038,
  0.022380045,
  -0.07762321,
  -0.021538403,
  -0.04092178,
  -0.011441483,
  0.022595994,
  -0.027228203,
  -0.0018961717,
  -0.008912174,
  0.101277374,
  0.011198262,
  0.10072374,
  0.03861099,
  0.02108267,
  -0.054758802,
  0.042814255,
  -0.021805428,
  -0.03708225,
  -0.031950336,
  -0.032654367,
  0.048815306,
  -0.0001778108,
  -0.025130456,
  0.009322101,
  -0.057608534,
  -0.07825605,
  0.04272

In [44]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1",
)

{'upserted_count': 20}

In [45]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}